In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import string
import math


In [2]:
df = pd.read_csv('tweets.csv')
stop_words = set(stopwords.words('english'))
positive_freq = Counter()
negative_freq = Counter()
neutral_freq = Counter()

for index, row in df.iterrows():
    tweet = row['tweets']
    label = row['label']
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(tweet)
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    if label == 1:
        positive_freq.update(filtered_tweet)
    elif label == -1:
        negative_freq.update(filtered_tweet)
    elif label == 0:
        neutral_freq.update(filtered_tweet)



results = []
for index, row in df.iterrows():
    tweet = row['tweets']
    label = row['label']
    
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(tweet)
    
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    bias = 0 if label == 0 else (1 if label == 1 else -1)
    
    pos_freq_sum = sum(positive_freq[word] for word in filtered_tweet)
    neg_freq_sum = sum(negative_freq[word] for word in filtered_tweet)
    neu_freq_sum = sum(neutral_freq[word] for word in filtered_tweet)

    results.append([bias, pos_freq_sum, neg_freq_sum])

print(results)
print("\n")
print(len(results))
print("\n")
print(positive_freq)
print("\n")
print(positive_freq)

[[-1, 29, 45], [1, 36, 26], [1, 87, 66], [1, 43, 21], [1, 31, 21], [-1, 0, 6], [1, 32, 22], [-1, 27, 29], [1, 54, 24], [1, 24, 19], [1, 62, 29], [1, 51, 30], [1, 26, 19], [1, 61, 13], [1, 49, 7], [-1, 35, 44], [-1, 32, 27], [1, 42, 6], [1, 51, 24], [-1, 36, 39], [1, 90, 54], [-1, 26, 45], [-1, 0, 8], [-1, 33, 54], [-1, 22, 33], [1, 54, 23], [-1, 34, 47], [-1, 23, 23], [-1, 67, 77], [-1, 0, 7], [1, 41, 23], [1, 54, 12], [-1, 37, 53], [1, 33, 1], [-1, 38, 40], [1, 22, 19], [1, 9, 0], [1, 37, 8], [-1, 44, 56], [-1, 25, 29], [1, 10, 0], [-1, 25, 34], [-1, 40, 34], [1, 31, 2], [1, 36, 5], [1, 49, 25], [1, 91, 49], [-1, 3, 3], [1, 60, 28]]


49


Counter({'modi': 22, 'vote': 10, 'india': 7, 'govt': 5, 'one': 5, 'modis': 5, '2019': 4, 'support': 4, 'job': 4, 'election': 3, 'like': 3, 'need': 3, 'ensure': 3, 'good': 3, 'anti': 3, 'via': 3, 'say': 2, 'bjp': 2, 'supporters': 2, 'chowkidar': 2, 'great': 2, 'become': 2, 'things': 2, 'gst': 2, 'castes': 2, 'would': 2, 'give': 2, '’': 2, 'people': 2

In [3]:
def prob(word, dict):
    numerator = dict[word]
    denominator = sum(dict[i]for i in dict)
    return (numerator / denominator)



In [4]:
# implementing laplacian smoothing
def prob_laplace(word, dict):
    numerator = dict[word] + 1
    denominator = sum(dict[i]for i in dict) + len(dict)
    return (numerator / denominator)

In [5]:
positive_tweets = 0
negative_tweets = 0
neutral_tweets = 0
for i in df['label']:
    if i == -1:
        negative_tweets = negative_tweets + 1
    elif i == 1:
        positive_tweets = positive_tweets + 1
    # else:
    #     negative_tweets = negative_tweets + 1

# P(1)
prob_positive = positive_tweets / (positive_tweets + negative_tweets)
print(prob_positive)
# P(-1)
prob_negative = negative_tweets / (positive_tweets + negative_tweets)
print(prob_negative)
# P(0)
# prob_neutral = neutral_tweets / (positive_tweets + negative_tweets)
print(prob_positive + prob_negative)

0.5714285714285714
0.42857142857142855
1.0


In [15]:
def naive_bayes (tweet, positive_dict, negative_dict, prob_positive, prob_negative):
    stop_words = set(stopwords.words('english'))
    lower_case=tweet.lower()
    clean=lower_case.translate(str.maketrans('','',string.punctuation))
    word_tokens=clean.split()
    filtered_tweet = [w for w in word_tokens if not w in stop_words]

    prob_rec=[]
    
    res=0
    for i in filtered_tweet:
        n = prob_laplace(i, positive_freq)
        d = prob_laplace(i, negative_freq)
        prob_rec.append(i)
        prob_rec.append(n)
        prob_rec.append(d)
        tmp = math.log(n/d)
        res = res + tmp
    p = math.log(prob_positive / prob_negative)
    return p + res
        

In [18]:
def predict (data, actual):
    right = 0
    total = 0
    for i in range (len(data)):
        if naive_bayes(data[i], positive_freq, negative_freq, prob_positive, prob_negative) > 0 and actual[i] == 1:
            # print(i)
            right = right + 1
            total = total + 1
        elif naive_bayes(data[i], positive_freq, negative_freq, prob_positive, prob_negative) < 0 and actual[i] == -1:
            # print(i)
            right = right + 1
            total = total + 1
        else:
            # print(i)
            total = total + 1
    print ("\n",right / total)

In [19]:
dataset = pd.read_csv('tweets.csv')
sentence = dataset["tweets"]
nature = dataset['label']

predict(sentence, nature)


 0.9795918367346939


In [41]:
def prob_res(positive_dict, negative_dict,func):
    p_sum=sum(positive_dict[item] for item in positive_dict)
    n_sum=sum(negative_dict[item] for item in positive_dict)
    prob_list=[]
    merg_dict = positive_dict | negative_dict
    for i in merg_dict:
        tmp=[]
        p_i=func(i,positive_dict)
        n_i=func(i,negative_dict)
        tmp.append(i)
        tmp.append(p_i)
        tmp.append(n_i)
        prob_list.append(tmp)
    
    return prob_list

        

In [45]:
a= prob_res(positive_freq, negative_freq, prob_laplace)
print(a)


[['talk', 0.0026666666666666666, 0.004106776180698152], ['nonsense', 0.0013333333333333333, 0.002053388090349076], ['continue', 0.0013333333333333333, 0.002053388090349076], ['drama', 0.0013333333333333333, 0.002053388090349076], ['vote', 0.015333333333333332, 0.014373716632443531], ['modi', 0.04066666666666666, 0.04106776180698152], ['say', 0.002, 0.002053388090349076], ['welcome', 0.0013333333333333333, 0.002053388090349076], ['bjp', 0.004666666666666667, 0.006160164271047228], ['told', 0.0013333333333333333, 0.002053388090349076], ['rahul', 0.0026666666666666666, 0.004106776180698152], ['main', 0.0013333333333333333, 0.002053388090349076], ['campaigner', 0.0013333333333333333, 0.002053388090349076], ['think', 0.0013333333333333333, 0.002053388090349076], ['relax', 0.0013333333333333333, 0.002053388090349076], ['asking', 0.0013333333333333333, 0.002053388090349076], ['supporters', 0.002, 0.002053388090349076], ['prefix', 0.0013333333333333333, 0.002053388090349076], ['chowkidar', 0.0